<a href="https://colab.research.google.com/github/sahiliem/colab/blob/main/Transformer_In_Detail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 25.1 MB/s eta 0:00:00


In [2]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

res = classifier("Feel good today")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [3]:
res

[{'label': 'POSITIVE', 'score': 0.999860405921936}]

In [17]:
# Inside Pipeline abstraction

from transformers import AutoTokenizer,AutoModelForSequenceClassification

model_name = "distilbert-base-uncased-finetuned-sst-2-english"

model_AutoModelForSequenceClassification = AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer_AutoTokenizer = AutoTokenizer.from_pretrained(model_name)






In [18]:
classifier2 = pipeline(task="sentiment-analysis",model=model_AutoModelForSequenceClassification,tokenizer=tokenizer_AutoTokenizer)

res2 = classifier2("Feel good today")

In [19]:
res2

[{'label': 'POSITIVE', 'score': 0.999860405921936}]

In [30]:
# Fine tuning AutoTokenizer Tokenizer, AutoModelForSequenceClassification model

sequence = "Using a Transformer network is simple"

tokens = tokenizer_AutoTokenizer.tokenize(sequence)

tokens

['using', 'a', 'transform', '##er', 'network', 'is', 'simple']

In [31]:
ids = tokenizer_AutoTokenizer.convert_tokens_to_ids(tokens)
ids


[2478, 1037, 10938, 2121, 2897, 2003, 3722]

In [32]:
decoded_string = tokenizer_AutoTokenizer.decode(ids)
decoded_string

'using a transformer network is simple'

In [33]:
#########################
X_train = ["I've been waiting for HuggingFace course my whole life",
           "Python is great!"]

res3=classifier2(X_train)
res3

[{'label': 'POSITIVE', 'score': 0.9861214756965637},
 {'label': 'POSITIVE', 'score': 0.9998615980148315}]

In [34]:
####
batch = tokenizer_AutoTokenizer(X_train,padding=True,truncation=True,max_length=512, return_tensors="pt" )
batch

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005, 17662, 12172,  2607,
          2026,  2878,  2166,   102],
        [  101, 18750,  2003,  2307,   999,   102,     0,     0,     0,     0,
             0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}

In [35]:
import torch
import torch.nn.functional as F

with torch.no_grad():
  outputs= model_AutoModelForSequenceClassification(**batch)
  print(outputs)
  predictions = F.softmax(outputs.logits,dim=1)
  print(predictions)
  labels = torch.argmax(predictions,dim=1)
  print(labels)





SequenceClassifierOutput(loss=None, logits=tensor([[-2.1236,  2.1398],
        [-4.2745,  4.6111]]), hidden_states=None, attentions=None)
tensor([[1.3878e-02, 9.8612e-01],
        [1.3835e-04, 9.9986e-01]])
tensor([1, 1])


In [36]:
save_directory = "savedmodel"
tokenizer_AutoTokenizer.save_pretrained(save_directory)
model_AutoModelForSequenceClassification.save_pretrained(save_directory)

In [37]:
# Load the saved model

tok_saved_model = AutoTokenizer.from_pretrained(save_directory)
model_saved_model = AutoModelForSequenceClassification.from_pretrained(save_directory)

In [38]:
# Run the saved model

classifier3 = pipeline(task="sentiment-analysis",model=model_saved_model,tokenizer=tok_saved_model)
res4 = classifier3("Feel good today")
res4

[{'label': 'POSITIVE', 'score': 0.999860405921936}]